In [226]:
# https://github.com/mm909/Predicting-Trump
    
import keras
import tensorflow as tf
from shutil import copyfile
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, LSTM, Dropout, GRU, Embedding, TimeDistributed, BatchNormalization, Bidirectional
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.optimizers import RMSprop, Adam

import import_ipynb
from MLEXPS.MLEXPS import *

## Data

In [229]:
file_name = input('TXT File Name: ')
text = open(file_name+'.txt', encoding="utf8").read()
print('corpus length:', len(text))

TXT File Name: todes
corpus length: 4293166


In [230]:
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print(f'unique chars: {len(chars)}')

SEQUENCE_LENGTH = 80
step = 4
sentences = []
next_chars = []
for i in range(0, len(text) - SEQUENCE_LENGTH, step):
    sentences.append(text[i: i + SEQUENCE_LENGTH])
    next_chars.append(text[i + SEQUENCE_LENGTH])
print(f'num training examples: {len(sentences)}')

X = np.zeros((len(sentences), SEQUENCE_LENGTH, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

print("X.shape:", X.shape)
print("y.shape:", y.shape)

unique chars: 63
num training examples: 1073272
X.shape: (1073272, 80, 63)
y.shape: (1073272, 63)


# Models

## Original model

In [ ]:
# Making Model
# model = load_model('D:/Predictive-Text/experiments/predictiveTrump/20200330-140314/weights/weights-improvement-04-0.6747.hdf5')

model = Sequential()

# CudDNNLSTM not working so I changed it to LSTM -> See following forum
# https://forums.developer.nvidia.com/t/importerror-cannot-import-name-cudnnlstm-from-tensorflow-keras-layers/82778/3
# model.add(CuDNNLSTM(len(chars) * 5, input_shape=(SEQUENCE_LENGTH, len(chars))))

model.add(LSTM(len(chars) * 5, input_shape=(SEQUENCE_LENGTH, len(chars))))

model.add(BatchNormalization())
model.add(Activation('selu'))

model.add(Dense(len(chars) * 2))
model.add(BatchNormalization())
model.add(Activation('selu'))

model.add(Dense(len(chars) * 2))
model.add(BatchNormalization())
model.add(Activation('selu'))

model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.001)
# reduce_lr = keras.callbacks.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.7, patience=2, min_lr=0.00001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


models = [model]
args = [{'x':X,
         'y':y,
         'batch_size':124,
         'epochs':5,
#          'epochs':50,
         'shuffle':False,
         'validation_split':0.05,}]
         # 'callbacks': [reduce_lr]

ml = MLEXPS()
ml.setTopic('todes')
ml.setCopyFileList(['train.ipynb'])
ml.setModels(models)
ml.setArgList(args)
ml.saveBestOnly = False
ml.startExprQ()

model.save('./models/todes_model2.h5')

# Model with dropout

In [227]:
# Making Model
# model = load_model('D:/Predictive-Text/experiments/predictiveTrump/20200330-140314/weights/weights-improvement-04-0.6747.hdf5')

model = Sequential()

# CudDNNLSTM not working so I changed it to LSTM -> See following forum
# https://forums.developer.nvidia.com/t/importerror-cannot-import-name-cudnnlstm-from-tensorflow-keras-layers/82778/3
# model.add(CuDNNLSTM(len(chars) * 5, input_shape=(SEQUENCE_LENGTH, len(chars))))

model.add(LSTM(len(chars) * 5, input_shape=(SEQUENCE_LENGTH, len(chars)),dropout=0.25))

model.add(BatchNormalization())
model.add(Activation('selu'))

model.add(Dense(len(chars) * 2))
model.add(BatchNormalization())
model.add(Activation('selu'))

model.add(Dense(len(chars) * 2))
model.add(BatchNormalization())
model.add(Activation('selu'))

model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.001)
# reduce_lr = keras.callbacks.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.7, patience=2, min_lr=0.00001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


models = [model]
args = [{'x':X,
         'y':y,
         'batch_size':124,
         'epochs':5,
#          'epochs':50,
         'shuffle':False,
         'validation_split':0.05,}]
         # 'callbacks': [reduce_lr]

ml = MLEXPS()
ml.setTopic('todes')
ml.setCopyFileList(['train.ipynb'])
ml.setModels(models)
ml.setArgList(args)
ml.saveBestOnly = False
ml.startExprQ()

model.save('./models/todes_model3.h5')

MLEXPS v3
Length of queue: 1
Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_26 (LSTM)               (None, 315)               477540    
_________________________________________________________________
batch_normalization_48 (Batc (None, 315)               1260      
_________________________________________________________________
activation_71 (Activation)   (None, 315)               0         
_________________________________________________________________
dense_55 (Dense)             (None, 126)               39816     
_________________________________________________________________
batch_normalization_49 (Batc (None, 126)               504       
_________________________________________________________________
activation_72 (Activation)   (None, 126)               0         
_________________________________________________________________
dense_56 (Dense)        

## Felipe's model

In [228]:

model = Sequential()

model.add(LSTM(len(chars) * 5, input_shape=(SEQUENCE_LENGTH, len(chars)), dropout=0.25))

model.add(BatchNormalization())
model.add(Activation('selu'))

model.add(Dense(len(chars) * 2))
model.add(BatchNormalization())
model.add(Activation('selu'))

model.add(Dense(len(chars) * 2))
model.add(BatchNormalization())
model.add(Activation('selu'))

model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


models = [model]
args = [{'x':X,
         'y':y,
         'batch_size':124,
         'epochs':5,
#          'epochs':50,
         'shuffle':False,
         'validation_split':0.05,}]
         # 'callbacks': [reduce_lr]

ml = MLEXPS()
ml.setTopic('todes')
ml.setCopyFileList(['train.ipynb'])
ml.setModels(models)
ml.setArgList(args)
ml.saveBestOnly = False
ml.startExprQ()


model.save('./models/todes_felipe_model.h5')

MLEXPS v3
Length of queue: 1
Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_27 (LSTM)               (None, 315)               477540    
_________________________________________________________________
batch_normalization_51 (Batc (None, 315)               1260      
_________________________________________________________________
activation_75 (Activation)   (None, 315)               0         
_________________________________________________________________
dense_58 (Dense)             (None, 126)               39816     
_________________________________________________________________
batch_normalization_52 (Batc (None, 126)               504       
_________________________________________________________________
activation_76 (Activation)   (None, 126)               0         
_________________________________________________________________
dense_59 (Dense)        

## felipe's other model

In [ ]:
# model = Sequential()


# model.add(Embedding(len(chars), 32, input_length=80))

# model.add(Bidirectional(LSTM(16, dropout=0.25, return_sequences=True)))


# model.add(TimeDistributed(Dense(len(chars))))
# model.add(Activation('softmax'))

# optimizer = Adam(lr=0.001)
# model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# model.fit(X, y, batch_size=128, epochs=5, verbose=1, shuffle=True, validation_split=0.2)
# model.save('felipe_2.h5')